# homuncu_loc data organisation 

In [1]:
import napari
from skimage import io
import btrack
import os
from tqdm.auto import tqdm
from homuncu_loc import dataio
import os, glob
import shutil
import pandas as pd
from collections import defaultdict


# Check to see how many images have sc analyses etc

In [2]:
root_dir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/'

In [3]:
results = dataio.sc_analysis_progress_check(root_dir)

In [4]:
results

,ID,expt_type,has_sc_analyses,sc_types,has_tif,has_vsi,has_metadata,image_location,image_fn
0,112,iAT1_iAT2,1,[_z_tracks_masks.h5],True,True,True,/run/user/30046150/gvfs/smb-share:server=data2...,20x_21-12-031B_A12456_Multichannel Z-Stack_202...
1,113,iAT1_iAT2,1,[_z_tracks_masks.h5],True,True,True,/run/user/30046150/gvfs/smb-share:server=data2...,20x_21-12-031B_A12456_Multichannel Z-Stack_202...
2,121,iAT1_iAT2,1,[_z_tracks_masks.h5],True,True,True,/run/user/30046150/gvfs/smb-share:server=data2...,20x_21-12-031B_A12456_Multichannel Z-Stack_202...
3,122,iAT1_iAT2,1,[_z_tracks_masks.h5],True,True,True,/run/user/30046150/gvfs/smb-share:server=data2...,20x_21-12-031B_A12456_Multichannel Z-Stack_202...
4,123,iAT1_iAT2,1,[_z_tracks_masks.h5],True,True,True,/run/user/30046150/gvfs/smb-share:server=data2...,20x_21-12-031B_A12456_Multichannel Z-Stack_202...
5,194,iAT1_iAT2,2,"[_z_tracks_masks.h5, _z_tracks_masks.csv]",True,True,True,/run/user/30046150/gvfs/smb-share:server=data2...,20x_21-12-029A_A3456_Multichannel Z-Stack_2022...
6,195,iAT1_iAT2,2,"[_z_tracks_masks.csv, _z_tracks_masks.h5]",True,True,True,/run/user/30046150/gvfs/smb-share:server=data2...,20x_21-12-029A_A3456_Multichannel Z-Stack_2022...
7,196,iAT1_iAT2,2,"[_z_tracks_masks.csv, _z_tracks_masks.h5]",True,True,True,/run/user/30046150/gvfs/smb-share:server=data2...,20x_21-12-029A_A3456_Multichannel Z-Stack_2022...
8,235,iAT1_iAT2,1,[_z_tracks_masks.h5],True,True,True,/run/user/30046150/gvfs/smb-share:server=data2...,20x_21-12-028A_A23456_Multichannel Z-Stack_202...
9,236,iAT1_iAT2,1,[_z_tracks_masks.h5],True,True,True,/run/user/30046150/gvfs/smb-share:server=data2...,20x_21-12-028A_A23456_Multichannel Z-Stack_202...


In [6]:
results['sc_types'].iloc[1]

['_z_tracks_masks_no_mtb_measure.h5',
 '_sc_results.csv',
 '_z_tracks_masks.h5',
 '_z_tracks_masks_d100.h5']

In [5]:
results.to_csv('/home/dayn/analysis/homuncu_loc/data_repository.csv')

### Create empty directory structure

In [6]:
# Specify the base directory where you want to create the structure
base_dir = "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/another_attempt_at_new_org"

# Call the function to create the structure
dataio.create_experiment_structure(base_dir)

print("Directory structure created successfully!")


Directory structure created successfully!


### Add files into directory structure using information from data organisation spreadsheet.

In [62]:
# Read the CSV data
csv_path = "/home/dayn/Downloads/Untitled spreadsheet - Sheet1.csv"
df = pd.read_csv(csv_path)

# Define the base directory where the files will be organized
source_dir = "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/old_organisation"
base_dir = "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/new_organisation"  # Directory for copied files

# Loop through the rows in the DataFrame
for index, row in tqdm(df.iterrows(), total = len(df)):
    filename = row["basename"].split('.')[0]
    markers = row["Markers"].replace('-', '_')
    n_cell_types = row["N cell types"]
    
    # Construct the source path by searching for the filename
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if filename in file:
                source_path = os.path.join(root, file)
                print(f'Moving source filename: {source_path}')
                if n_cell_types == 2:
                    if ('.h5' in file) or ('.hdf5' in file) or ('.csv' in file) or ('.xlsx' in file) or ('.prism' in file): 
                        target_dir = os.path.join(base_dir, "iAT1_iAT2_experiments", markers, 'sc_analyses')
                    else:
                        target_dir = os.path.join(base_dir, "iAT1_iAT2_experiments", markers, 'images')
                elif n_cell_types == 4:
                    if ('.h5' in file) or ('.hdf5' in file) or ('.csv' in file) or ('.xlsx' in file) or ('.prism' in file): 
                        target_dir = os.path.join(base_dir, "iAT1_iAT2_iVEC_macrophage_experiments", markers, 'sc_analyses')
                    else:
                        target_dir = os.path.join(base_dir, "iAT1_iAT2_iVEC_macrophage_experiments", markers, 'images')
                else:
                    if ('.h5' in file) or ('.hdf5' in file) or ('.csv' in file) or ('.xlsx' in file) or ('.prism' in file): 
                        target_dir = os.path.join(base_dir, "iAT1_iAT2_iVEC_experiments", markers, 'sc_analyses')
                    else:
                        target_dir = os.path.join(base_dir, "iAT1_iAT2_iVEC_experiments", markers, 'images')

               # Create the target directory if it doesn't exist
                os.makedirs(target_dir, exist_ok=True)

                # create destination path 
                dest_path = os.path.join(target_dir, file)

                if os.path.exists(dest_path):
                    continue
                
                try:# Copy the file to the target directory
                    shutil.move(source_path, dest_path)
                except:
                    print(f'Error: {source_path}')
                
        # now copy directories that match the original filename
        for dir in dirs:
            if filename in dir:
                source_path = os.path.join(root, dir)
                print(f'Moving source directory: {source_path}')
                if n_cell_types == 2:
                    target_dir = os.path.join(base_dir, "iAT1_iAT2_experiments", markers, 'images')
                elif n_cell_types == 4:
                    # if "macrophage" in markers:
                    target_dir = os.path.join(base_dir, "iAT1_iAT2_iVEC_macrophage_experiments", markers, 'images')
                else:
                    target_dir = os.path.join(base_dir, "iAT1_iAT2_iVEC_experiments", markers, 'images')
    
        
                # Create the target directory if it doesn't exist
                os.makedirs(target_dir, exist_ok=True)

                # create destination path 
                dest_path = os.path.join(target_dir, dir)
                if os.path.exists(dest_path):
                    continue
                try:
                    # Copy the file to the target directory
                    shutil.move(source_path, dest_path)
                except:
                    print(f'Error: {source_path}')


  0%|          | 0/77 [00:00<?, ?it/s]

Moving source filename: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/old_organisation/results_(old_organisation, currently reviewing)/run0/DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295_z_tracks_masks.h5
Moving source filename: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/old_organisation/Job_1/DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295.vsi
Moving source filename: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/old_organisation/Job_1/DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295.tif
Moving source filename: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on C

### 

### Check to see if all files transferred.

In [2]:
# Define the base directory where the files will be organized
dir1 = "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/old_organisation"
dir2 = "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/new_organisation"  # Directory for copied files


_, _, missing_files = dataio.compare_directories_recursive(dir1, dir2)
print("Files in the first directory that are missing in the second directory:")
for file in missing_files:
    print(file)
    print('')



Files in the first directory that are missing in the second directory:


### Check to see if any files were corrupted in the transfer

Might happen if the connection times out or if the image files weren't properly transferred in the first place

In [3]:

# Example usage
directory_to_search = "/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis"
empty_files_list = dataio.find_empty_files(directory_to_search)

for file_path in empty_files_list:
    print(f"Empty file found: {file_path}")


Empty file found: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/images/20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1299.vsi
Empty file found: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/images/20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1300.vsi
Empty file found: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/images/20230705_40X_23-01-001B3_Multichannel Z-Stack_20230705_1298.vsi
Empty file found: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_exper

### Check to see what images have what analyses in them.

In [13]:
import os

def analyze_images(root_dir="homuncu_loc_image_analysis"):
    """
    Analyze .tif images to check for corresponding .vsi images and metadata directories.

    Parameters:
    - root_dir (str): The root directory to start the search from.

    Returns:
    - list: A list of dictionaries containing details about each .tif image.
    """

    results = []

    # Traverse the specified directory structure
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.tif'):
                tif_path = os.path.join(root, file)
                tif_basename = os.path.splitext(file)[0]
                tif_ID = tif_basename.split('_')[-1].split('.tif')[0]

                # Check for corresponding .vsi file
                vsi_exists = os.path.exists(os.path.join(root, tif_basename + '.vsi'))

                # Check for corresponding metadata directory
                metadata_dir = os.path.join(os.path.dirname(root), tif_basename)
                metadata_exists = os.path.isdir(metadata_dir) and any(f for f in os.listdir(metadata_dir) if f.startswith('metadata'))

                results.append({
                    "tif_path": tif_path,
                    "tif_basename": tif_basename,
                    "has_vsi": vsi_exists,
                    "has_metadata": metadata_exists
                })

    results = pd.DataFrame(data = results)

    return results
